In [3]:
%load_ext autoreload
%autoreload 2

from datasets import Zone1
ds = Zone1('data/zone1_prior_load.parquet')
ds.data.head()

from validation import walkforward, Error
from sklearn.ensemble import RandomForestRegressor
import numpy as np

n = 50
for s in [n*0.1 for n in range(1, 6)]:
    model = RandomForestRegressor(n_estimators=n, max_samples=s, n_jobs=4)
    (predictions, errors) = walkforward(model, ds.data, ds.data[ds.actual],
                                        ds.validation_start, ds.validation_end, Zone1.next_hour,
                                        ds.features)
    overall_error = Error(ds.validation_data[ds.actual], np.concatenate(predictions))
    print(overall_error)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Error(mae=521.3571310730592, max=3452.2881999999972, total=4567088.468199993)
Error(mae=529.0425043150685, max=3630.9405999999963, total=4634412.337800025)
Error(mae=533.5468552739725, max=3437.9390000000003, total=4673870.45220002)
Error(mae=525.5292052054795, max=3574.656399999998, total=4603635.837599989)
Error(mae=531.8591386757989, max=3597.7415999999994, total=4659086.054799996)


In [2]:
overall_error

Error(mae=525.7232836431832, max=3594.14957142857, total=4605335.9647143185)

In [7]:
cols = ds.data.columns.to_series()
cols = cols.drop([ds.actual, ds.mtlf])
for n in [10, 50, 100]:
    model = RandomForestRegressor(n_estimators=n, max_samples=0.1, n_jobs=4)
    (predictions, errors) = walkforward(model, ds.data, ds.data[ds.actual],
                                        ds.validation_start, ds.validation_end, Zone1.next_hour,
                                        cols)
    overall_error = Error(ds.validation_data[ds.actual], np.concatenate(predictions))
    print(overall_error)

Error(mae=594.9278533105023, max=3995.833999999999, total=5211567.9949999945)
Error(mae=577.7956147260273, max=3512.7947999999997, total=5061489.585000002)
Error(mae=574.1588173744294, max=3618.3545999999988, total=5029631.240199989)


In [5]:
overall_error = Error(ds.validation_data[ds.actual], np.concatenate(predictions))
print(overall_error)

Error(mae=586.3442405936072, max=3470.216000000004, total=5136375.547599995)


### Conclusion

Including all available features does not help the regressor, but smaller sampling does help generalization. However, we are still doing worse than the naive model and much worse than the baseline MTLF.